In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain.llms import HuggingFacePipeline
# from langchain.prompts import PromptTemplate
# from langchain.chains import LLMChain
from transformers import pipeline

model_path = "h:/ML_Models/_gemma/model/gemma-2b-it"  # local folder

tokenizer = AutoTokenizer.from_pretrained(model_path)

model = AutoModelForCausalLM.from_pretrained(model_path)
print("Model loaded.")

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512)
llm = HuggingFacePipeline(pipeline=pipe)

print("Pipeline loaded.")

c:\personal\_python\fastapi\PetshopAPIPython\langchain_test\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:55<00:00, 27.74s/it]
Device set to use cpu


Model loaded.


C:\Users\admin\AppData\Local\Temp\ipykernel_22192\3728606683.py:15: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


Pipeline loaded.


In [2]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_milvus import Milvus

# Embedding model
embedding_model = HuggingFaceEmbeddings(model_name="multi-qa-mpnet-base-dot-v1")

# Connect to your existing collection
vector_store = Milvus(
    embedding_model,
    collection_name="Dogs_Breeds_milvus_EN_1",
    connection_args={"host": "localhost", "port": "19530"}
)
print("Vector store loaded.")

C:\Users\admin\AppData\Local\Temp\ipykernel_22192\1625853733.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="multi-qa-mpnet-base-dot-v1")


Vector store loaded.


In [ ]:
query_criteria = "dogs for rescue operations"
results = vector_store.similarity_search(
    query_criteria,
    k=5,
)
print(f"Retrieved {len(results)} documents from Milvus for the query: '{query_criteria}'")


Retrieved 3 documents from Milvus for the query: 'dogs for rescue operations'


In [ ]:
# Use in LangChain RAG
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS
retriever = FAISS.from_documents(results, embedding=embedding_model).as_retriever()
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, chain_type="stuff", )

question = "Given the following documents, which docs are good for rescue operations?"

response = qa_chain.run(question)

print(response)
print("Response generated.")

C:\Users\admin\AppData\Local\Temp\ipykernel_22192\2779256704.py:9: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain.run(question)


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

GERMAN SHEPHERD Nationality: Germany. Origin: Known for its intelligence and learning ability, the German Shepherd has long been used as a guard dog, police dog, and guide dog—not for herding, as its name suggests. Description: Deep chest, not too wide. Body is deep but not long, though longer than its height at the withers. Well-formed ribcage, straight and muscular back. Broad, strong flanks. Tail is thick, reaching the hock, hanging and slightly curved at rest. Raised in motion but never above the back line. Medium-sized, almond-shaped eyes, preferably dark brown, with lively, intelligent, confident expression. Medium ears, broad at base, high-set, triangular, held upright and slightly forward. Head proportionate to body, moderately broad between ears. Strong muzzle, straight nasal bridge, firm lips, strong incisors with 

In [8]:
#Answers using prompt
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

custom_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
    You are a rescue operations expert. Based on the following documents, identify which ones are most relevant for rescue missions.

    Documents:
    {context}

    Question:
    {question}

    Answer in a clear, concise manner. If none are relevant, say so explicitly.
    """
)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",  # or "map_reduce", "refine" if you want more control
    chain_type_kwargs={"prompt": custom_prompt}
)

# Run the chain
question = "Given the following documents, which docs are good for rescue operations?"
response = qa_chain.run(question)
print(response)
print("Response with prompt generated.")


    You are a rescue operations expert. Based on the following documents, identify which ones are most relevant for rescue missions.

    Documents:
    GERMAN SHEPHERD Nationality: Germany. Origin: Known for its intelligence and learning ability, the German Shepherd has long been used as a guard dog, police dog, and guide dog—not for herding, as its name suggests. Description: Deep chest, not too wide. Body is deep but not long, though longer than its height at the withers. Well-formed ribcage, straight and muscular back. Broad, strong flanks. Tail is thick, reaching the hock, hanging and slightly curved at rest. Raised in motion but never above the back line. Medium-sized, almond-shaped eyes, preferably dark brown, with lively, intelligent, confident expression. Medium ears, broad at base, high-set, triangular, held upright and slightly forward. Head proportionate to body, moderately broad between ears. Strong muzzle, straight nasal bridge, firm lips, strong incisors with scissor bi